In [1]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bilinmiyor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bilinmiyor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bilinmiyor\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
df= pd.read_csv("../data/stemmed_sentences.csv")

In [6]:
df

,ntc sensor short circuit open circuit heater short circuit heater continu oper continu water intak error water drainag error motor triac short circuit water level sensor error motor open circuit aqua stop failur rins sensor ntc short circuit open circuit heater short circuit heater continu oper valv short circuit pump drain water motor short circuit water level sensor faulti pump short circuit door lock open circuit door lock short circuit motor wind open circuit aqua stop failur motor modul commun cabl faulti jet pump open circuit flow meter open short circuit rins sensor short circuit laundri remain foami unbalanc load machin safe water drainag level activ commun cabl fault lcd board main board door switch work load sensor work commun cabl fault control panel main board main board written read level sensor control flow meter control main voltag limit ntc short circuit open circuit air duct open flap close commun sensor dri heater open circuit overh drum overh condens inlet overh condens outlet ntc short circuit open circuit heater short circuit heater
0,continu oper valv triac short circuit pump dra...
1,triac short circuit water level sensor faulti ...
2,door lock open circuit door triac short circui...
3,circuit water sourc error water temperatur sen...
4,level error water drainag error water suppli e...
...,...
141,lean load calcul cross check rang perform pro...
142,lean acceler fuel deliveri system malfunct hig...
143,sensor signal rang oxygen sensor circuit bank ...
144,voltag seri throttl control error needl lift s...


In [7]:
df.columns = ["0"]

# NaN değerleri ve boş stringleri temizle
df = df.dropna()
df = df[df["0"].str.strip() != ""]

# Temizlenmiş ilk 5 cümleyi göster
print(df.head())

                                                   0
0  continu oper valv triac short circuit pump dra...
1  triac short circuit water level sensor faulti ...
2  door lock open circuit door triac short circui...
3  circuit water sourc error water temperatur sen...
4  level error water drainag error water suppli e...


In [8]:
#Varsayalım ki DataFrame'iniz df ve açıklama '0' sütununda
text = ' '.join(df['0'].astype(str).tolist())  # Tüm yorumları tek bir metin haline getir
sentences = sent_tokenize(text)  # Metni cümlelere ayır

In [9]:
# Stopwords listesini almak
stop_words = set(stopwords.words('english'))

In [10]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize

# Lemmatizer ve Stemmer nesnelerini oluştur
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
# Kelimeleri tokenleştirip, lemmatize etme ve stemleme
def preprocess_sentence(sentence):
    tokens = word_tokenize(sentence)
    filtered_tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    return lemmatized_tokens, stemmed_tokens

# Kullanım örneği
tokenized_corpus_lemmatized = []
tokenized_corpus_stemmed = []

for sentence in sentences:
    lemmatized_tokens, stemmed_tokens = preprocess_sentence(sentence)
    tokenized_corpus_lemmatized.append(lemmatized_tokens)
    tokenized_corpus_stemmed.append(stemmed_tokens)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
# Ön işlenmiş token listelerini tekrar metne çeviriyoruz
lemmatized_texts = [' '.join(tokens) for tokens in tokenized_corpus_lemmatized]

lemmatized_texts[:3]


['continu oper valv triac short circuit pump drain water motor triac short circuit water level sensor faulti pump short circuit door lock open circuit door triac short circuit motor triac open circuit water sourc error water temperatur sensor error water level error water drainag error water suppli error electr error water temperatur high water temperatur low water heat error door open water fill error water pressur error water leakag error elev height limit exceed door open attempt move door open elev move overload condit brake releas brake system error jerk occur sudden stop vibrat deceler stop elev stop move error occur control unit power suppli error elev elev oper excess high temperatur cabin door lock system faulti driver fan error intern temperatur error control suppli error input phase error capacitor voltag error output phase error driver overh error dc overvoltag error ac overvoltag error dc overcurr error ac overcurr error brake error encod error low dc voltag error line fre

In [13]:
# TF-IDF vektörizerı başlatıyoruz
vectorizer = TfidfVectorizer()

# TF-IDF matrisini oluşturuyoruz
#terim frekansları, belge frekanslarıni hesplar
#TF-IDF vektörlerine dönüştürür
tfidf_matrix = vectorizer.fit_transform(lemmatized_texts)

## Kelimeleri alalım
#F-IDF vektörleştirme işleminde kullanılan tüm kelimelerin essiz bir listesini döndürur
feature_names = vectorizer.get_feature_names_out()

# TF-IDF matrisini pandas DataFrame'e çevir-gorunurluk acisindan- calismasi kolay
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# İlk birkaç satırı gösterelim-ilk 5 cümle
print(tfidf_df.head())

#Her satır bir cümleyi temsil eder
#Her sütun bir kelimeyi temsil eder
#Hücreler ise o kelimenin o cümledeki TF-IDF skorudur - her cumle icin degisir-bakiniz:slaytlar

         ac   acceler    actuat     addit    affect      air   airflow  \
0  0.010919  0.005459  0.005459  0.005459  0.038216  0.09827  0.005459   

      alarm    altern     ammet  ...    vehicl    ventil    vibrat    voltag  \
0  0.065513  0.005459  0.005459  ...  0.005459  0.005459  0.005459  0.114648   

      volum     water       way    window      work    yellow  
0  0.005459  0.114648  0.021838  0.005459  0.021838  0.005459  

[1 rows x 289 columns]


In [14]:
tfidf_df.to_csv('tfidf_stemmed.csv', index=False)